In [ ]:
import pandas as pd
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scripts import morphoGrid as morph
from scipy.spatial import cKDTree

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Get the sundaland shelf coordinates

In [ ]:
shelfXYZ = "./data/sundaland_shelf.csv"
pd_shelf = pd.read_csv(
            shelfXYZ,
            sep=r"	",
            engine="c",
            na_filter=False,
            dtype=np.float,
            low_memory=False,
        )

shelfXY = pd_shelf.values[:, 0:2]
shelfZ = pd_shelf.values[:, 2]
tree = cKDTree(shelfXY)

# Read a given badlands time step

In [ ]:
badland_topo = morph.morphoGrid(folder='lowsub/h5', bbox = None, dx=5000)
badland_topo.loadHDF5(timestep=1000)

In [ ]:
regX = badland_topo.x.flatten()
regY = badland_topo.y.flatten()
regXY = np.vstack([regX, regY]).T

## Define sundaland shelf values

In [ ]:
# This only needs to be done once!
dists, inds = tree.query(regXY, k=1)
shelfIDs = np.where(dists<=5500)[0]


sundaIDs = np.zeros(len(regY))
sundaIDs[shelfIDs] = 1
viewIDs = sundaIDs.reshape(badland_topo.x.shape)

shelfPtsNb = len(shelfIDs)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = plt.gca()
im = ax.imshow(viewIDs, cmap ='seismic', vmin = -1, vmax = 1, 
                    interpolation ='nearest', origin ='lower') 
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)
plt.show()

# fig.savefig('shelfvals.pdf')

Get number of points on the shelf above sea level

In [ ]:
aboveIDs = np.where(badland_topo.z.flatten()[shelfIDs]>badland_topo.sl)[0]

In [ ]:
percAerial = len(aboveIDs)/shelfPtsNb

In [ ]:
percAerial

# Looping over timesteps

In [ ]:
low_percent = []
for step in range(1001):
    
    if step % 100 == 0:
        print('Processing step:',step)
        
    badland_topo = morph.morphoGrid(folder='lowsub/h5', bbox = None, dx=5000)
    badland_topo.loadHDF5(timestep=step)
    aboveIDs = np.where(badland_topo.z.flatten()[shelfIDs]>badland_topo.sl)[0]
    percAerial = len(aboveIDs)/shelfPtsNb
    low_percent.append(percAerial)

In [ ]:
mid_percent = []
for step in range(1001):
    
    if step % 100 == 0:
        print('Processing step:',step)
        
    badland_topo = morph.morphoGrid(folder='midsub/h5', bbox = None, dx=5000)
    badland_topo.loadHDF5(timestep=step)
    aboveIDs = np.where(badland_topo.z.flatten()[shelfIDs]>badland_topo.sl)[0]
    percAerial = len(aboveIDs)/shelfPtsNb
    mid_percent.append(percAerial)

In [ ]:
high_percent = []
for step in range(1001):
    
    if step % 100 == 0:
        print('Processing step:',step)
        
    badland_topo = morph.morphoGrid(folder='highsub/h5', bbox = None, dx=5000)
    badland_topo.loadHDF5(timestep=step)
    aboveIDs = np.where(badland_topo.z.flatten()[shelfIDs]>badland_topo.sl)[0]
    percAerial = len(aboveIDs)/shelfPtsNb
    high_percent.append(percAerial)

### Save in file

In [ ]:
time = np.arange(len(percent))

df = pd.DataFrame(
    {
        "time": time-1000.,
        "low": low_percent,
        "mid": mid_percent,
        "high": high_percent,
    }
)

df.to_csv(
    "shelfExpose.csv",
    columns=["time", "low", "mid", "high"],
    sep=",",
    index=False,
)

# Plot results

The plot is done in notebook 6


Read file

In [ ]:
file = "shelfExpose.csv"
data = pd.read_csv(
            file,
            sep=r",",
            engine="c",
            na_filter=False,
            dtype=np.float,
            low_memory=False,
        )

time = data.values[:, 0]
low = data.values[:, 1]*100.
mid = data.values[:, 2]*100.
high = data.values[:, 3]*100.

View data

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10,5))

plt.plot(time,low,color='k', ls='-', lw=1.5)
plt.plot(time,mid,color='b', ls='-', lw=1.5)
plt.plot(time,high,color='r', ls='-', lw=1.5)

# # plt.grid()
plt.xlim(-1000,0)
plt.ylim(0,110)
plt.xlabel('Time (kyr BP)')
plt.ylabel('Percentage of exposed sundaland shelf')
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10,5))

plt.plot(time,low,color='k', ls='-', lw=1.5)
plt.plot(time,mid,color='b', ls='-', lw=1.5)
plt.plot(time,high,color='r', ls='-', lw=1.5)

# # plt.grid()
plt.xlim(-450,0)
plt.ylim(0,110)
plt.xlabel('Time (kyr BP)')
plt.ylabel('Percentage of exposed sundaland shelf')
fig.tight_layout()

In [ ]:
# plt.figure(figsize=(10, 8))
# ax = plt.gca()
# im = ax.imshow(badland_topo.z, cmap ='seismic', vmin = -1000, vmax = 1000, 
#                     interpolation ='nearest', origin ='lower') 
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)

# plt.colorbar(im, cax=cax)
# plt.show()